In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle
import tensorflow as tf
from tensorflow import keras

In [26]:
import pickle
from keras.models import load_model

In [27]:
# load model , scaler pickle , one hot 

model = load_model('model.h5')

#load encoder and scaler
with open('geo_encoder.pkl','rb') as file:
    geo_encoder = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [28]:
input_data = {
    "CreditScore": 502,
    "Geography": "France",
    "Gender": "Female",
    "Age": 42,
    "Tenure": 8,
    "Balance": 159660.8,
    "NumOfProducts": 3,
    "HasCrCard": 1,
    "IsActiveMember": 0,
    "EstimatedSalary": 113931.57
}


In [29]:
# input_data = {
#     "CreditScore": 350,              # Low credit score
#     "Geography": "Germany",          # Geography encoded (e.g., Germany)
#     "Gender": "Female",              # Gender encoded (e.g., Female)
#     "Age": 29,                       # Young age, less stable
#     "Tenure": 1,                     # New customer, high churn risk
#     "Balance": 500,                  # Low balance, less engagement
#     "NumOfProducts": 1,              # Only one product, low engagement
#     "HasCrCard": 0,                  # No credit card, less engagement
#     "IsActiveMember": 0,             # Inactive member, likely to churn
#     "EstimatedSalary": 30000.0       # Low salary, potentially less engaged
# }


In [30]:
#one hot encoding
geography_encoder = geo_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geography_encoder,columns=geo_encoder.get_feature_names_out(['Geography']))

d:\LLM folder\ANN_implemntation\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [31]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [32]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,502,France,Female,42,8,159660.8,3,1,0,113931.57


In [33]:
#combine one-hot value with input value
input_data = pd.concat([input_data.reset_index(drop=True),geo_encoded_df],axis=1)

In [34]:
input_data = input_data.drop('Geography',axis=1)

In [35]:
#Encode Categorical variables
input_data['Gender']=label_encoder_gender.transform(input_data['Gender'])

In [36]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,502,0,42,8,159660.8,3,1,0,113931.57,1.0,0.0,0.0


In [44]:
#scaling the data
input_scaler_data = scaler.transform(input_data)

In [45]:
input_scaler_data

array([[-1.5530028 , -1.09499335,  0.29493847,  1.03972843,  1.33785683,
         2.53355998,  0.64920267, -1.02583358,  0.23472492,  1.00150113,
        -0.57946723, -0.57638802]])

Prediction

In [46]:
prediction = model.predict(input_scaler_data)
prediction[0][0]

1/1 [==============================] - 1s 708ms/step


0.98734206

In [47]:
if prediction > 0.5 :
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is likely to churn.
